# GraphRAG for Python

In this module, you will setup and use the [Neo4j GraphRAG for Python package](https://neo4j.com/docs/neo4j-graphrag-python/current/?_gl=1*8wl1t6*_gcl_au*MjEzNTI4NjkxNy4xNzU3MjU4NDMzLjc4MDQ1OTczLjE3NTg0MTY3NjUuMTc1ODQxNjc2NA..*_ga*MTkzMzgxNTk1LjE3NTcyNTg0MzQ.*_ga_DL38Q8KGQC*czE3NjMyNjQ0MzMkbzQ3JGcxJHQxNzYzMjczNTg2JGo0NSRsMCRoMA..*_ga_DZP8Z65KK4*czE3NjMyNjQ0MzMkbzQ3JGcxJHQxNzYzMjczNTg2JGo0NSRsMCRoMA..) to:

- Create and configure a vector retriever.

- Perform semantic search using a vector index.

- Build a GraphRAG pipeline that uses retrievers to give context to an LLM.

- Use graph traversal in combination with vector search for enhanced retrieval.

- Implement text-to-Cypher retrieval for natural language querying.

The [GraphRAG for Python](https://neo4j.com/docs/neo4j-graphrag-python/current/) package (neo4j-graphrag) allows you to access Neo4j Generative AI functions including:

- Retrievers

- GraphRAG pipelines

- Knowledge graph construction

The purpose is to provide a first party package to developers, where Neo4j can guarantee long term commitment and maintenance as well as being fast to ship new features and high performing patterns and methods.

You will use the neo4j-graphrag package to create retrievers and implement simple applications that use GraphRAG to provide context to LLM queries.

You must set up a development environment to run the code examples and exercises.


# Vector Retriever

In this lesson, you will create a vector retriever to retrieve relevant data from Neo4j.

A retriever is a component that takes unstructured data (typically a users query) and retrieves relevant data.

You will create a vector retriever that find similar movies based on a movie plot. The retriever will use the moviePlots vector index you used to search for similar movies using Cypher.

To find similar movies using a retriever you need to:

1. Connect to a Neo4j database

2. Create an embedder to convert users queries into vectors

3. Create a retriever that uses the moviePlots vector index

4. Use the retriever to search for similar movies using the users query

5. Parse the results


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from neo4j import GraphDatabase

# Connect to Neo4j database
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USERNAME"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

driver.verify_connectivity()

## Embedder

Create the embedder that will convert the user’s query into a vector:

In [2]:
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

# Create embedder
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")

## Retriever

Create the retriever that will use the moviePlots vector index:

In [3]:
from neo4j_graphrag.retrievers import VectorRetriever

# Create retriever
retriever = VectorRetriever(
    driver,
    index_name="moviePlots",
    embedder=embedder,
    return_properties=["title", "plot"],
)

## Search

You can use the retriever to search the vector index by passing a query and the number of results to return. The retriever will use the embedder to convert the query into a vector to use in the search.

Search for similar movies:

In [4]:
# Search for similar items
result = retriever.search(query_text="Toys coming alive", top_k=5)

The search method returns a list of items that match the query.

Iterate over the items and print the results:

In [5]:
# Parse results
for item in result.items:
    print(item.content, item.metadata["score"])

{'title': 'Toy Story', 'plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room."} 0.9173871278762817
{'title': 'Jumanji', 'plot': 'When two kids find and play a magical board game, they release a man trapped for decades in it and a host of dangers that can only be stopped by finishing the game.'} 0.9108158349990845
{'title': 'Powder', 'plot': 'A young bald albino boy with unique powers shakes up the rural community he lives in.'} 0.9069517850875854
{'title': 'White Balloon, The (Badkonake sefid)', 'plot': "Several people try to take advantage of a little girl's innocence to hustle money her mom gave to her to buy a goldfish with."} 0.9009029269218445
{'title': 'Indian in the Cupboard, The', 'plot': 'On his ninth birthday a boy receives many presents. Two of them first seem to be less important: an old cupboard from his brother and a little Indian figure made of plastic from his best ...'} 0.9008167386054993


In [6]:
driver.close()